In [1]:
library(caret, quiet = TRUE)
library(base64enc)
library(httr, quiet = TRUE)

library(mlbench)


Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:

## multiclass classification in iris dataset:

data(BostonHousing)
BostonHousing$chas = as.numeric(BostonHousing$chas)

set.seed(1960)

dataset = BostonHousing[, -14] 

create_model  =  function() {

    model <- train(medv ~ ., data = BostonHousing, method = "xgbTree")    

    return(model)
}


In [3]:
model = create_model()
# cat(model$feature_names)
# print(model)

In [4]:
pred_labels <- predict(model, BostonHousing[, -14] , type="raw")
df = data.frame(BostonHousing[,14])
names(df) = c("medv")
df$Estimator = pred_labels
df$Error = df$Estimator - df$medv
MAPE = mean(abs(df$Error / df$medv))
summary(df)
MAPE

      medv         Estimator          Error          
 Min.   : 5.00   Min.   : 4.856   Min.   :-3.256257  
 1st Qu.:17.02   1st Qu.:17.041   1st Qu.:-0.537835  
 Median :21.20   Median :21.241   Median :-0.001406  
 Mean   :22.53   Mean   :22.524   Mean   :-0.008950  
 3rd Qu.:25.00   3rd Qu.:25.344   3rd Qu.: 0.518806  
 Max.   :50.00   Max.   :50.347   Max.   : 2.894483  

[1] 0.03629602

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "xgboost_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json", verbose())
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)










































































WITH "XGB_0" AS 
(WITH "DT_node_lookup" AS 
(SELECT "ADS"."KEY" AS "KEY", CASE WHEN ("ADS"."Feature_5" < 6.79699993) THEN CASE WHEN ("ADS"."Feature_0" < 9.28069496) THEN CASE WHEN ("ADS"."Feature_5" < 6.14300013) THEN 7 ELSE 8 END ELSE CASE WHEN ("ADS"."Feature_0" < 59.5282974) THEN 9 ELSE 10 END END ELSE CASE WHEN ("ADS"."Feature_5" < 7.43700027) THEN CASE WHEN ("ADS"."Feature_11" < 130.345001) THEN 11 ELSE 12 END ELSE 6 END END AS node_id_2 
FROM "INPUT_DATA" AS "ADS"), 
"DT_node_data" AS 
(SELECT "Values".nid AS nid, "Values"."Estimator" AS "Estimator" 
FROM (SELECT 6 AS nid, 12.9722738 AS "Estimator" UNION ALL SELECT 7 AS nid, 5.41195965 AS "Estimator" UNION ALL SELECT 8 AS nid, 6.82309532 AS "Estimator" UNION ALL SELECT 9 AS nid, 3.57075024 AS "Estimator" UNION ALL SELECT 10 AS nid, 1.28000009 AS "Estimator" UNION ALL SELECT 11 AS nid, 2.0400002 AS "Estimator" UNION ALL SELECT 12 AS nid, 9.28163338 AS "Estimat

# Execute the SQL Code

In [7]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [8]:
df_sql = dataset
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

# df_sql

In [9]:
colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

[1] "Feature_0"  "Feature_1"  "Feature_2"  "Feature_3"  "Feature_4" 
 [6] "Feature_5"  "Feature_6"  "Feature_7"  "Feature_8"  "Feature_9" 
[11] "Feature_10" "Feature_11" "Feature_12" "KEY"

In [10]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out[order(df_sql_out$KEY),])

,KEY,Estimator
374,1,24.16700
84,2,21.79294
168,3,33.37066
56,4,34.18503
320,5,35.72997
64,6,28.43761


In [11]:
# df_sql_out

# R XGBoost Output

In [12]:
estimator  =  predict(model, dataset, type = "raw")
df_r_out = data.frame(estimator)
names(df_r_out) = c("Estimator")

df_r_out$KEY = seq.int(nrow(dataset))
head(df_r_out)


Estimator,KEY
24.16700,1
21.79294,2
33.37067,3
34.18502,4
35.72997,5
28.43762,6


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Estimator_1,Estimator_2
1,24.16700,24.16700
2,21.79294,21.79294
3,33.37067,33.37066
4,34.18502,34.18503
5,35.72997,35.72997
6,28.43762,28.43761


In [25]:
df_merge$Error = df_merge$Estimator_1 - df_merge$Estimator_2
df_merge$AbsError = abs(df_merge$Error)
head(df_merge)


KEY,Estimator_1,Estimator_2,Error,AbsError
1,24.16700,24.16700,-1.937272e-06,1.937272e-06
2,21.79294,21.79294,1.634829e-06,1.634829e-06
3,33.37067,33.37066,1.209642e-05,1.209642e-05
4,34.18502,34.18503,-4.507608e-06,4.507608e-06
5,35.72997,35.72997,6.338102e-06,6.338102e-06
6,28.43762,28.43761,2.674194e-06,2.674194e-06


In [35]:
df_merge_largest_errors = df_merge[df_merge$AbsError > 0.0001,]
df_merge_largest_errors

KEY,Estimator_1,Estimator_2,Error,AbsError


In [34]:
stopifnot(nrow(df_merge_largest_errors) == 0)

In [17]:
summary(df_sql_out)

      KEY          Estimator     
 Min.   :  1.0   Min.   : 4.856  
 1st Qu.:127.2   1st Qu.:17.041  
 Median :253.5   Median :21.241  
 Mean   :253.5   Mean   :22.524  
 3rd Qu.:379.8   3rd Qu.:25.344  
 Max.   :506.0   Max.   :50.347  

In [18]:
summary(df_r_out)

   Estimator           KEY       
 Min.   : 4.856   Min.   :  1.0  
 1st Qu.:17.041   1st Qu.:127.2  
 Median :21.241   Median :253.5  
 Mean   :22.524   Mean   :253.5  
 3rd Qu.:25.344   3rd Qu.:379.8  
 Max.   :50.347   Max.   :506.0  

In [36]:
summary(df_merge)

      KEY         Estimator_1      Estimator_2         Error           
 Min.   :  1.0   Min.   : 4.856   Min.   : 4.856   Min.   :-1.231e-05  
 1st Qu.:127.2   1st Qu.:17.041   1st Qu.:17.041   1st Qu.:-3.400e-06  
 Median :253.5   Median :21.241   Median :21.241   Median :-7.782e-07  
 Mean   :253.5   Mean   :22.524   Mean   :22.524   Mean   :-1.885e-07  
 3rd Qu.:379.8   3rd Qu.:25.344   3rd Qu.:25.344   3rd Qu.: 1.857e-06  
 Max.   :506.0   Max.   :50.347   Max.   :50.347   Max.   : 2.555e-05  
    AbsError        
 Min.   :3.366e-09  
 1st Qu.:1.184e-06  
 Median :2.553e-06  
 Mean   :3.692e-06  
 3rd Qu.:4.682e-06  
 Max.   :2.555e-05  